In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
### Importing the dataset using pandas 

df = pd.read_csv("loan (1).csv")

In [3]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
df.shape

(614, 13)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [6]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
df.dropna(inplace = True)

In [8]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [9]:
df.shape

(480, 13)

### After dropping the null value loosing somewhere around 22% of the data --> so continuing with the dropped null value dataset 

In [10]:
### Seprating my dependent and independent variable 

X = df.drop(["Loan_Status","Loan_ID"],axis=1)
Y = df["Loan_Status"]

In [11]:
### Checking catagorical values features 

X[["Gender","Married","Education","Self_Employed","Property_Area"]].nunique()

Gender           2
Married          2
Education        2
Self_Employed    2
Property_Area    3
dtype: int64

In [12]:
X.shape

(480, 11)

In [13]:
X["Dependents"].unique()

array(['1', '0', '2', '3+'], dtype=object)

In [14]:
X['Dependents'] = df['Dependents'].replace('3+', 4)

In [15]:
X["Dependents"].unique()

array(['1', '0', '2', 4], dtype=object)

In [16]:
### Dependent column contatins numerical value but as a  dtype its obj column so convertiin dependents column into int value 

df['Dependents'] = pd.to_numeric(df['Dependents'], errors='coerce')
df['Dependents'] = df['Dependents'].fillna(0).astype(int)

In [17]:
### Seprating catagorical and numerical column 

cat_cols = ["Gender","Married","Education","Self_Employed","Property_Area"]
num_cols = ["Dependents","ApplicantIncome","CoapplicantIncome","LoanAmount","Loan_Amount_Term","Credit_History"]

In [18]:
cat_cols

['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']

In [19]:
num_cols

['Dependents',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

In [20]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [21]:
### Genrating ohe and after that with feature converting it into df for further processing

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', drop="first")
cat_X = pd.DataFrame(ohe.fit_transform(X[cat_cols]))
cat_X.columns = ohe.get_feature_names_out(input_features=cat_cols)

In [22]:
print(cat_X.shape)
cat_X.head()

(480, 6)


,Gender_Male,Married_Yes,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,1.0,1.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,1.0,0.0,1.0
2,1.0,1.0,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,1.0
4,1.0,1.0,0.0,1.0,0.0,1.0


In [23]:
### Scaling the numerical columns data 

scaler = StandardScaler()
num_X = pd.DataFrame(scaler.fit_transform(X[num_cols]))
print(num_X.shape)

(480, 6)


In [24]:
### Concating catagorical column after applying OHE with numerical column after applying standard scaler 

cat_X = cat_X.values
num_X = num_X.values
X = pd.concat([pd.DataFrame(cat_X), pd.DataFrame(num_X)], axis=1)

In [25]:
X.shape
X = np.asarray(X)

In [26]:
### Applying label encode in dependet variable 

from sklearn.preprocessing import LabelEncoder
LabelEncoder = LabelEncoder()
Y  = LabelEncoder.fit_transform(Y)

In [27]:
### Spliting the data 

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [28]:
### Importing libraries for further model processing 

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [29]:
### Crearting Class for every ml model 

LogisticRegression = LogisticRegression()
SVC = SVC()
DecisionTreeClassifier = DecisionTreeClassifier()
RandomForestClassifier = RandomForestClassifier()

In [30]:
### Importing Randomized Search Cv 

from sklearn.model_selection import RandomizedSearchCV

In [31]:
### Crearing hyper parameter for every ml model

logistic_param = {"penalty": ["l1","l2"], "C": [1,2,10], "max_iter" : [100,200], "solver": ["saga","liblinear"]
             ,"multi_class":["auto", "ovr", "multinomial"],"verbose": [0,1]},
SVC_param = {'C': [ 1,3, 10], "kernel":["linear","poly","sigmoid"],'gamma': ["scale"],"class_weight":["balanced"]}
decision_tree_param ={"criterion":["gini","entropy"], "max_depth":[None]+[2,10,15] ,"min_samples_split":[2,10,15,20] , "min_samples_leaf":[3,6,9,12],
                                "max_features" :["auto","sqrt", "log2"]},
random_forest__param ={"n_estimators":[20,50,100] , "criterion":["gini","entropy","log_loss"],"max_depth":[None]+[10,20,30], "min_samples_split":np.arange(2,10),
                              "min_samples_leaf":np.arange(2,10), "max_features" :["auto","sqrt", "log2"]}

In [32]:
LR_model = RandomizedSearchCV(LogisticRegression, logistic_param,cv=5,n_iter=10,random_state=0)
SVC_model = RandomizedSearchCV(SVC, SVC_param,cv=15,n_iter=12,random_state=5)
DT_model = RandomizedSearchCV(DecisionTreeClassifier, decision_tree_param,cv=15,n_iter=15,random_state=33)
RF_model = RandomizedSearchCV(RandomForestClassifier, random_forest__param,cv=18,n_iter=20,random_state=42)


In [33]:
LR_model_search1 = LR_model.fit(X_train,Y_train)
SVC_model_search2 = SVC_model.fit(X_train,Y_train)
DT_model_search3 = DT_model.fit(X_train,Y_train)
RF_model_search3 = RF_model.fit(X_train,Y_train)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]convergence after 84 epochs took 0 seconds
convergence after 97 epochs took 0 seconds
convergence after 78 epochs took 0 seconds
convergence after 77 epochs took 0 seconds
convergence after 40 epochs took 0 seconds
max_iter reached after 0 seconds
max_iter reached after 0 seconds
max_iter reached after 0 seconds
max_iter reached after 0 seconds


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

convergence after 84 epochs took 0 seconds
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


convergence after 146 epochs took 0 seconds
convergence after 168 epochs took 0 seconds
convergence after 143 epochs took 0 seconds
convergence after 125 epochs took 0 seconds
convergence after 80 epochs took 0 seconds
[LibLinear]

In [34]:
print(f"Logistic Regression best parameter is {LR_model_search1.best_params_} & its best score is {LR_model_search1.best_score_}")
print(f"SVC best parameter is {SVC_model_search2.best_params_} & its best score is {SVC_model_search2.best_score_}")
print(f"Decision Tress model  best parameter is {DT_model_search3.best_params_} & its best score is {DT_model_search3.best_score_}")
print(f"Random Forest best parameter is {RF_model_search3.best_params_} & its best score is {RF_model_search3.best_score_}")

Logistic Regression best parameter is {'verbose': 1, 'solver': 'liblinear', 'penalty': 'l2', 'multi_class': 'auto', 'max_iter': 100, 'C': 1} & its best score is 0.8067788461538463
SVC best parameter is {'kernel': 'linear', 'gamma': 'scale', 'class_weight': 'balanced', 'C': 1} & its best score is 0.812987012987013
Decision Tress model  best parameter is {'min_samples_split': 20, 'min_samples_leaf': 12, 'max_features': 'sqrt', 'max_depth': None, 'criterion': 'entropy'} & its best score is 0.76002886002886
Random Forest best parameter is {'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 7, 'max_features': 'auto', 'max_depth': None, 'criterion': 'gini'} & its best score is 0.8168119099491649


## As per the above result , Best Score achived by SVC & Random Forest ..

## Logistic Regression Result ----> 80.67% 
## SVC Result --------------------> 81.29% 
## Decision Tree -----------------> 76.30%
## Random Forest -----------------> 81.37%